In [3]:
import numpy as np
import pandas as pd
import os
import json

# Load the CSV file
csv_file_path = "../data/20230914_processed_table.csv"
data = pd.read_csv(csv_file_path)

# Ensure relevant columns are properly formatted as strings
data["diff"] = data["diff"].astype(str)
data["sum"] = data["sum"].astype(str)
data["diff_std"] = data["diff_std"].astype(str)
data["sum_std"] = data["sum_std"].astype(str)

# Extract the unique wavelengths from the FILTER01 column
wavelengths = data["FILTER01"].unique()

# Define the sub-wavelengths for "Open" configurations
open_wavelengths = [610, 670, 720, 760]

# Prepare output directory
output_dir = os.path.dirname(csv_file_path)

# Extract unique HWP and IMR angles
HWP_angs = data["RET-POS1"].unique()
IMR_angs = data["D_IMRANG"].unique()

# Loop through each unique wavelength
for wavelength in wavelengths:
    wavelength_mask = data["FILTER01"] == wavelength
    wavelength_data = data[wavelength_mask]

    if wavelength.lower() == "open":
        for open_wavelength_idx, open_wavelength in enumerate(open_wavelengths):
            configs_for_wavelength = []
            double_diffs = []
            double_sums = []
            double_diff_stds = []
            double_sum_stds = []

            for HWP_ang in HWP_angs:
                for IMR_ang in IMR_angs:
                    # Create masks for FLC states A and B
                    mask_A = (wavelength_data["RET-POS1"] == HWP_ang) & \
                             (wavelength_data["D_IMRANG"] == IMR_ang) & \
                             (wavelength_data["U_FLC"] == "A")

                    mask_B = (wavelength_data["RET-POS1"] == HWP_ang) & \
                             (wavelength_data["D_IMRANG"] == IMR_ang) & \
                             (wavelength_data["U_FLC"] == "B")

                    # Extract values for both FLC states
                    diff_A_list = wavelength_data[mask_A]["diff"].values
                    diff_B_list = wavelength_data[mask_B]["diff"].values
                    sum_A_list = wavelength_data[mask_A]["sum"].values
                    sum_B_list = wavelength_data[mask_B]["sum"].values
                    diff_std_A_list = wavelength_data[mask_A]["diff_std"].values
                    diff_std_B_list = wavelength_data[mask_B]["diff_std"].values
                    sum_std_A_list = wavelength_data[mask_A]["sum_std"].values
                    sum_std_B_list = wavelength_data[mask_B]["sum_std"].values

                    # Check if enough elements exist
                    if len(diff_A_list) > open_wavelength_idx and len(diff_B_list) > open_wavelength_idx:
                        try:
                            # Convert string values to list of floats (splitting on arbitrary whitespace)
                            diff_A = float(diff_A_list[open_wavelength_idx].strip("[]").split()[0])
                            diff_B = float(diff_B_list[open_wavelength_idx].strip("[]").split()[0])
                            sum_A = float(sum_A_list[open_wavelength_idx].strip("[]").split()[0])
                            sum_B = float(sum_B_list[open_wavelength_idx].strip("[]").split()[0])
                            diff_std_A = float(diff_std_A_list[open_wavelength_idx].strip("[]").split()[0])
                            diff_std_B = float(diff_std_B_list[open_wavelength_idx].strip("[]").split()[0])
                            sum_std_A = float(sum_std_A_list[open_wavelength_idx].strip("[]").split()[0])
                            sum_std_B = float(sum_std_B_list[open_wavelength_idx].strip("[]").split()[0])

                            # Compute normalized double differences and sums
                            unnormalized_double_diff = np.median([diff_A]) - np.median([diff_B])
                            unnormalized_double_sum = np.median([diff_A]) + np.median([diff_B])
                            total_sum = np.median([sum_A]) + np.median([sum_B])

                            normalized_double_diff = unnormalized_double_diff / total_sum
                            normalized_double_sum = unnormalized_double_sum / total_sum

                            unnormalized_double_diff_std = np.sqrt(diff_std_A ** 2 + diff_std_B ** 2)
                            unnormalized_double_sum_std = np.sqrt(diff_std_A ** 2 + diff_std_B ** 2)
                            total_sum_std = np.sqrt(sum_std_A ** 2 + sum_std_B ** 2)

                            double_diff_std = np.sqrt((unnormalized_double_diff_std / unnormalized_double_diff) ** 2 + 
                                                      (total_sum_std / total_sum) ** 2) * normalized_double_diff
                            
                            double_sum_std = np.sqrt((unnormalized_double_sum_std / unnormalized_double_sum) ** 2 + 
                                                     (total_sum_std / total_sum) ** 2) * normalized_double_sum

                            # Append results to lists
                            double_diffs.append(normalized_double_diff)
                            double_sums.append(normalized_double_sum)
                            double_diff_stds.append(double_diff_std)
                            double_sum_stds.append(double_sum_std)

                            # Generate system configuration dictionary
                            system_dict = {
                                "components": {
                                    "parang_rot": {"type": "Rotator", "properties": {"pa": 0}, "tag": "on-sky"},
                                    "m3": {"type": "DiattenuatorRetarder", "properties": {"phi": 2 * np.pi * 0, "epsilon": 0.03}, "tag": "on-sky"},
                                    "alt_rot": {"type": "Rotator", "properties": {"pa": 0}, "tag": "on-sky"},
                                    "lp": {"type": "LinearPolarizer", "properties": {"theta": 0}, "tag": "internal"},
                                    "hwp": {"type": "Retarder", "properties": {"phi": 2 * np.pi * HWP_ang, "delta_theta": 0}, "tag": "internal"},
                                    "image_rotator": {"type": "Retarder", "properties": {"phi": IMR_ang, "delta_theta": 0}, "tag": "internal"},
                                    "optics": {"type": "DiattenuatorRetarder", "properties": {"phi": 2 * np.pi * 0, "epsilon": 0.03}, "tag": "on-sky"},
                                    "flc": {"type": "Retarder", "properties": {"phi": 0, "theta": 0}, "tag": "internal"},
                                    "dichroic": {"type": "Retarder", "properties": {"phi": 2 * np.pi * 0, "theta": 0}, "tag": "internal"},
                                    "wollaston": {"type": "WollastonPrism", "properties": {"beam": "o", "transmission_ratio": 0.5}, "tag": "internal"},
                                },
                                "order": [
                                    "parang_rot", "m3", "alt_rot", "lp", "hwp",
                                    "image_rotator", "optics", "flc", "dichroic", "wollaston"
                                ],
                            }
                            configs_for_wavelength.append(system_dict)

                        except Exception as e:
                            print(f"Error processing row: {row}, Exception: {e}")
                            continue

            # Save data
            json_file_path = os.path.join(output_dir, f"instrument_configurations_{open_wavelength}.json")
            with open(json_file_path, "w") as json_file:
                json.dump(configs_for_wavelength, json_file, indent=4)

            np.save(os.path.join(output_dir, f"double_diffs_{open_wavelength}.npy"), np.array(double_diffs))
            np.save(os.path.join(output_dir, f"double_sums_{open_wavelength}.npy"), np.array(double_sums))
            np.save(os.path.join(output_dir, f"double_diff_stds_{open_wavelength}.npy"), np.array(double_diff_stds))
            np.save(os.path.join(output_dir, f"double_sum_stds_{open_wavelength}.npy"), np.array(double_sum_stds))

            print(f"Saved configurations and metrics for open wavelength {open_wavelength}")


Saved configurations and metrics for open wavelength 610
Saved configurations and metrics for open wavelength 670
Saved configurations and metrics for open wavelength 720
Saved configurations and metrics for open wavelength 760
